In [525]:
import re
import pandas as pd
import numpy as np
import time
import os
import Image
import pytesseract
import pylab as pl
import matplotlib.pyplot as plt
from selenium import webdriver
import scipy.stats as ss
from selenium.webdriver.common.keys import Keys
from time import mktime
from datetime import datetime
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [438]:
driver = webdriver.Firefox()
pagina='http://www.reclameaqui.com.br/ranking/'
driver.get(pagina)

In [417]:
driver = webdriver.Firefox()
pagina='http://www.reclameaqui.com.br/ranking/'
driver.get(pagina)
time.sleep(5)
driver.get(driver.find_element_by_xpath('//*[@id="tabela-ranking"]/tbody/tr[14]/td/div/table[1]/tbody/tr/td/a').get_attribute("href"))
time.sleep(5)
driver.get(driver.find_element_by_xpath('//*[@id="reclamacoes-empresa"]/div/div[2]/div/div/ul/li/a').get_attribute("href"))
time.sleep(5)
text = driver.find_element_by_xpath('//*[@id="reclamacoes-empresa"]/div/div[2]/div/div[2]/ul/li[1]/span[2]').text
print text
#driver.get(driver.find_element_by_xpath('//*[@id="todo-conteudo-resultado"]/div/div/center/div/ul/li[7]/a').get_attribute("href"))
#time.sleep(5)
#breakCaptha(driver)
#time.sleep(5)
#driver.get('http://www.google.com')
#driver.quit()

KeyboardInterrupt: 

In [103]:
driver.find_element_by_class_name("dados-reclamacao-fanpage").text

u'Domingo, 05 de Abril de 2015 - 11:19, Ribeir\xe3o Preto - SP'

In [439]:
def scrapRanking(arg):
    try:
        for i in range(20):
            arg['business'][i]=driver.find_element_by_xpath('//*[@id="tabela-ranking"]/tbody/tr[14]/td/div/table['+str(i+1)+']/tbody/tr/td/a').get_attribute("title")
            arg['href'][i]=driver.find_element_by_xpath('//*[@id="tabela-ranking"]/tbody/tr[14]/td/div/table['+str(i+1)+']/tbody/tr/td/a').get_attribute("href")
    except:
        return arg

In [440]:
ranking=pd.DataFrame(np.zeros((20,2)))
ranking.columns=['business','href']
scrapRanking(ranking)

/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [441]:
meses = [u'Janeiro', u'Fevereiro', u'Março', u'Abril', u'Maio', u'Junho', u'Julho', u'Agosto', u'Setembro', u'Outubro', u'Novembro', u'Dezembro']

In [442]:
def setDate(txt):
    try:
        for i in range(12):
            txt = txt.replace(meses[i],u'0'+str(i+1))
            txt = txt.replace(u'de', '-')
            match = re.search(r'\d{2} - \d{2} - \d{4} - \d{2}:\d{2}', txt)
        return match, txt
    except:
        print 'Is not a String object'

In [443]:
def getDate(arg):
    match,txt = setDate(arg)
    date = time.strptime(match.group().replace(" ",""), '%d-%m-%Y-%H:%M')
    return date

In [444]:
def getPlace(arg):
    match, arg = setDate(arg)
    arg = arg.replace(match.group(),"")
    m = re.match(r"(\w+)", arg).group(0)
    arg = arg.replace(m,"")
    match = re.search(r', , ', arg)
    arg = arg.replace(match.group(),"")
    return arg

In [445]:
def breakCaptha(wd):
    try:
        oldtab = wd.current_window_handle
        body = wd.find_element_by_tag_name("body")
        body.send_keys(Keys.CONTROL + 't')
        wd.get('http://www.reclameaqui.com.br/indices/lista_reclamacoes/captcha.php')
        #You have to create the folder /captcha
        wd.save_screenshot("../marcim/captcha/screenshot.png")
        wd.find_element_by_tag_name("body").send_keys(Keys.ALT + Keys.NUMPAD1)
        wd.switch_to_window(oldtab)
        captcha = pytesseract.image_to_string(Image.open('../marcim/captcha/screenshot.png'), config="-psm 6")
        cp = driver.find_element_by_name('captcha')
        cp.send_keys(captcha)
        wd.find_element_by_xpath('//*[@id="reclamacoes-empresa"]/div/div[2]/div/div[2]/ul/div/div/form/input[2]').click()
    except:
        print "Didn't broke the captcha"

In order to gain performance, we prefer to get a list of URLs in third level of scraping, and then visit them. This already give us O(n³), if we had choose to scrap the fourth level, we would get a O(n⁴) of complexity order.

In [539]:
def thirdLevelScrap(i):
    try:
        subsubtable = pd.DataFrame(np.zeros((20,4)))
        subsubtable.columns=['business','url','date','place']
        for j in range(20):
            subsubtable['business'][j] = ranking['business'][i]
            subsubtable['url'][j] = driver.find_element_by_xpath('//*[@id="reclamacoes-empresa"]/div/div[2]/div/div[2]/ul/li['+str(j+1)+']/h3/a').get_attribute("href")
            text = driver.find_element_by_xpath('//*[@id="reclamacoes-empresa"]/div/div[2]/div/div[2]/ul/li['+str(j+1)+']/span[2]').text
            subsubtable['place'][j] = getPlace(text)
            subsubtable['date'][j] = datetime.datetime(*getDate(text)[:6])
        return subsubtable
    except:
            #Break the captcha and goes on
            print 'error, possibly a captcha asking'
            breakCaptha(driver)
            subsubtable = thirdLevelScrap(i)
            return subsubtable

In [545]:
def secondLevelScrap(i):
    try:
        k=2015
        subtable = pd.DataFrame(np.zeros((1,4)))
        subtable.columns=['business','url','date','place']
        while k>4:
            subsubtable = thirdLevelScrap(i);
            k = subsubtable['date'][subsubtable['date'].size-1].day
            print k
            #section to concatenate leafs of a business
            pieces=[subtable,subsubtable]
            subtable=pd.concat(pieces)
            print subtable['date'][subtable['date'].size-1].day
            nextPage = driver.find_element_by_xpath('//*[@id="todo-conteudo-resultado"]/div/div/center/div/ul/li[7]/a').get_attribute("href")
            driver.get(nextPage)
            time.sleep(5)
        return subtable
    except:
        print "An exception screw everything on second level, returning whatever you got"
        return subtable

In [464]:
def firstLevelScrap():
    try:
        table = pd.DataFrame(np.zeros((20,4)))
        table.columns=['business','url','date','place']
        for i in range(1):#ranking['href'].size):
                    driver.get(ranking['href'][i])
                    time.sleep(5)
                    #Take and get the url "todas as reclamações"
                    key = driver.find_element_by_xpath('//*[@id="reclamacoes-empresa"]/div/div[2]/div/div/ul/li/a').get_attribute("href")
                    driver.get(key)
                    subtable = secondLevelScrap(i)
                    #section to concatenate business
                    pieces=[table,subtable]
                    table=pd.concat(pieces)
        return table
    except:
        print "An exception screw everything on first level, returning whatever you got"
        return table

In [546]:
driver = webdriver.Firefox()
driver.get(ranking['href'][1])
key = driver.find_element_by_xpath('//*[@id="reclamacoes-empresa"]/div/div[2]/div/div/ul/li/a').get_attribute("href")
driver.get(key)
k=secondLevelScrap(1)

/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


5
An exception screw everything on second level, returning whatever you got


/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [522]:
subsubtable = pd.DataFrame(np.zeros((20,4)))
subsubtable.columns=['col1','col2','col3','col4']
#subsubtable['col2'].to_datetime()#df.Y*10000+df.M*100+df.D,format='%Y%m%d')
#subsubtable['col2'][3]=getDate(u'Domingo, 05 de Abril de 2015 - 11:19, Ribeir\xe3o Preto - SP')
subsubtable['col2'] = pd.to_datetime(subsubtable['col2'], format= "%d/%m/%Y/%H:%M")
print subsubtable

ValueError: time data 0.0 does not match format '%d/%m/%Y/%H:%M' (match)

In [541]:
date = getDate(u'Domingo, 05 de Abril de 2015 - 11:19, Ribeir\xe3o Preto - SP')
dt = datetime.datetime(*date[:6])
dt.day

5

In [401]:
ranking2=pd.DataFrame(np.zeros((1,2)))
ranking2.columns=['business','href']
pieces=[ranking,ranking2]
ranking2 = pd.concat(pieces)

In [450]:
print ranking[:1].get_values

<bound method DataFrame.get_values of                                     business  \
0  Net Serviços (tv, Banda Larga e Telefone)   

                                                href  
0  http://www.reclameaqui.com.br/indices/902/net-...  >


In [373]:
df = pd.DataFrame(np.random.randn(10, 4))
df.columns=['business','url','date','place']

In [374]:
df

,business,url,date,place
0,-0.061974,1.022713,2.232056,-0.543028
1,-2.281711,0.754404,2.620714,1.159897
2,0.312740,-0.909827,0.380729,-0.502907
3,-0.849179,-0.171854,-1.243311,-1.332489
4,1.234445,-2.380747,-0.574468,0.236022
5,-1.450001,-0.164806,0.020823,0.300747
6,-0.537417,-1.104948,1.790365,0.352829
7,-0.758693,-0.607587,0.013375,-2.052865
8,0.209966,-0.714054,0.511221,0.254405
9,-0.630053,0.838657,0.035113,0.686276


In [387]:
pieces = [df[:2], df[3:5], df]

In [388]:
concatenated = pd.concat(pieces)

In [389]:
concatenated

,business,url,date,place
0,-0.061974,1.022713,2.232056,-0.543028
1,-2.281711,0.754404,2.620714,1.159897
3,-0.849179,-0.171854,-1.243311,-1.332489
4,1.234445,-2.380747,-0.574468,0.236022
0,-0.061974,1.022713,2.232056,-0.543028
1,-2.281711,0.754404,2.620714,1.159897
2,0.312740,-0.909827,0.380729,-0.502907
3,-0.849179,-0.171854,-1.243311,-1.332489
4,1.234445,-2.380747,-0.574468,0.236022
5,-1.450001,-0.164806,0.020823,0.300747
